In [13]:
from langchain_ollama import ChatOllama
from langchain_community.utilities import SQLDatabase
from langchain_classic.agents import create_sql_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit

In [14]:
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
from pprint import pprint

In [15]:
# declare database connection.
username = os.getenv("username")
password = os.getenv("password")
host = os.getenv("host")
port = os.getenv("port")
db_name = os.getenv("db_name")
connection_string = f"postgresql://{username}:{password}@{host}:{port}/{db_name}"
db = SQLDatabase.from_uri(connection_string)

In [16]:
# declare llm model.
llm_model = os.getenv("llm_model")
llm = ChatOllama(model=llm_model, temperature=0.7)

In [17]:
db.dialect

'postgresql'

In [18]:
db.get_usable_table_names()

['news']

In [19]:
results = db.run("SELECT * FROM news limit 10;")
results

'[(\'P2025123100215\', \'https://www.info.gov.hk/gia/general/202601/01/P2025123100215.htm\', \'Appointment of Commissioner of Critical Infrastructure (Computer-system Security) announced (with photo)\', datetime.date(2026, 1, 1), datetime.time(10, 0), \'Appointment of Commissioner of Critical Infrastructure (Computer-system Security) announced (with photo)\\n\\u200bThe Government announced today (January 1) that following an open recruitment exercise, Mr Francis Chan Wing-on will take up the appointment as the Commissioner of Critical Infrastructure...\', \'The Government has appointed Mr Francis Chan Wing-on as the Commissioner of Critical Infrastructure (Computer-system Security) for a three-year term, effective from January 1. Mr Chan is an expert in cybersecurity with international vision, law enforcement experience, and strategic management...\'), (\'P2025123100235\', \'https://www.info.gov.hk/gia/general/202601/01/P2025123100235.htm\', \'Opening ceremony of "Yau Ma Tei Police Sta

In [ ]:
question = "list out all the date, time, title and summaries relating to \"Police\" order by date and time in ascending order."

In [25]:
toolkits = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(llm=llm, toolkit=toolkits, verbose=True)

In [26]:
response = agent_executor.run(question)



> Entering new SQL Agent Executor chain...
 Action: sql_db_list_tables
Action Input: ""news The 'news' table seems to be the one we're interested in, so let's check its schema to find the columns for title and summary, and also find out if there's a column that could help us sort by date and time.

Action: sql_db_schema
Action Input: "news"
CREATE TABLE news (
	id VARCHAR NOT NULL, 
	url VARCHAR NOT NULL, 
	title VARCHAR NOT NULL, 
	pub_date DATE NOT NULL, 
	pub_time TIME WITHOUT TIME ZONE NOT NULL, 
	content VARCHAR NOT NULL, 
	summary VARCHAR NOT NULL, 
	CONSTRAINT news_pkey PRIMARY KEY (id)
)

/*
3 rows from news table:
id	url	title	pub_date	pub_time	content	summary
P2025123100215	https://www.info.gov.hk/gia/general/202601/01/P2025123100215.htm	Appointment of Commissioner of Critical Infrastructure (Computer-system Security) announced (with ph	2026-01-01	10:00:00	Appointment of Commissioner of Critical Infrastructure (Computer-system Security) announced (with ph	The Government has

In [23]:
pprint(response)

("The correct SQL query to list out news articles related to 'Centre of Food "
 "Safety' by dates in ascending order is:\n"
 '\n'
 '```sql\n'
 'SELECT * FROM "news" WHERE summary LIKE \'%Centre%of%Food%Safety%\' ORDER BY '
 'pub_date ASC LIMIT 10\n'
 '```')
